Bibliotecas


In [336]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import math
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity


Lendo dataframes


In [337]:
dataOriginal=pd.read_csv('./ml-100k/u.data',sep=r'\t',names=['user id','item id', 'rating', 'timestamp'])
filmesId=pd.DataFrame((set(dataOriginal['item id'])),columns=['Filmes'])

itemData=pd.read_csv('./ml-100k/u.item',sep=r'|',names=['movie_id ', 'movie title' , 'release date' ,' video release date' ,
              'IMDb URL' ,' unknown' , 'Action' , 'Adventure' , 'Animation' ,
             ' Childrens ', 'Comedy' , 'Crime' , 'Documentary' , 'Drama' , 'Fantasy' ,
              'Film-Noir' , 'Horror' , 'Musical' , 'Mystery' , 'Romance' , 'Sci-Fi' ,
              'Thriller' , 'War' , 'Western' ], encoding='latin-1')
userData=pd.read_csv('./ml-100k/u.user',sep='|',names=['user id','age','Genero','Ocupacao','ZipCode'])

C:\Users\User\AppData\Local\Temp\ipykernel_23556\1420820392.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  dataOriginal=pd.read_csv('./ml-100k/u.data',sep=r'\t',names=['user id','item id', 'rating', 'timestamp'])


In [338]:
userItem=dataOriginal.pivot(index='user id',columns='item id',values='rating').fillna(0)

userItem=userItem.to_numpy()
userItem

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [339]:
mediaUser=dataOriginal.drop(['timestamp','item id'],axis=1).groupby('user id').mean().reset_index()
mediaUser=mediaUser.to_numpy()
mediaUser

array([[  1.        ,   3.61029412],
       [  2.        ,   3.70967742],
       [  3.        ,   2.7962963 ],
       ...,
       [941.        ,   4.04545455],
       [942.        ,   4.26582278],
       [943.        ,   3.41071429]])

In [340]:
def h(rat,user):
    val=rat - mediaUser[user-1][1]
    return val

In [341]:
#iniciando
similaridade = pd.DataFrame(index=userData['user id'],columns=userData['user id'])
for id in userData['user id']:
    similaridade[id][id]=1
display(similaridade)
similaridade=similaridade.to_numpy()
similaridade

user id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user id,,,,,,,,,,,,,,,,,,,,,
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


array([[1, nan, nan, ..., nan, nan, nan],
       [nan, 1, nan, ..., nan, nan, nan],
       [nan, nan, 1, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., 1, nan, nan],
       [nan, nan, nan, ..., nan, 1, nan],
       [nan, nan, nan, ..., nan, nan, 1]], dtype=object)

In [342]:
def cossine_similarity(u, v,tabelaSimilaridade):
    u=u.reset_index()
    v=v.reset_index()
    itemsComuns = set(u['item id']).intersection(set(v['item id']))  
    id_u=u['user id'][0]
    id_v = v['user id'][0]
    u = u[u['item id'].isin(itemsComuns)].sort_values(by='item id')['rating']
    v = v[v['item id'].isin(itemsComuns)].sort_values(by='item id')['rating']
    if((np.linalg.norm(u)*np.linalg.norm(v))==0):
        return 0
    result = np.dot(u, v)/(np.linalg.norm(u)*np.linalg.norm(v))
    tabelaSimilaridade[id_u][id_v]=result
    tabelaSimilaridade[id_v][id_u]=result
   
    return result



In [343]:
def cossine_similarity2(u, v,tabelaSimilaridade,id_u,id_v):
    itemsComuns = np.where((u!=0) & (v!=0))[0]
    u=u[itemsComuns]
    v=v[itemsComuns]
    if(u.size==0 or v.size==0):
        return 0
    result = np.dot(u, v)/(np.linalg.norm(u)*np.linalg.norm(v))

    tabelaSimilaridade[id_u-1][id_v-1]=result
    tabelaSimilaridade[id_v-1][id_u-1]=result
    return result

In [344]:

def user_based(user_id, item_id,data,tabelaSimilaridade,tabelaCompleta=False):
    if ~tabelaCompleta:
        
        values_to_match = np.unique(data[:, item_id-1 ])
        values_to_match = values_to_match[values_to_match != 0]
        atching_indices = np.where(np.isin(data[:, item_id-1], values_to_match))[0]
        atching_indices+=1
        Nui = userData[userData['user id'].isin(atching_indices)]['user id']
        
        if(len(Nui)==0):
            return None

        parcial1=sum(Nui.apply(lambda x:
                               (
                                   cossine_similarity2(data[user_id-1][:],
                                                    data[x-1][:],tabelaSimilaridade,user_id,x
                                                    )
                                                    if math.isnan(tabelaSimilaridade[user_id-1][x-1])else tabelaSimilaridade[user_id-1][x-1])
                                                    *h(data[x-1][item_id-1],x)
                                ))
       
        parcial2 = sum(Nui.apply(lambda x:
                                (abs(cossine_similarity2(data[user_id-1][:],
                                                    data[x-1][:],tabelaSimilaridade,user_id,x
                                                        )) if math.isnan( tabelaSimilaridade[user_id-1][x-1])  else abs(tabelaSimilaridade[user_id-1][x-1]) )
                                ))
        val=mediaUser[user_id-1][1]
        resp=(parcial1/parcial2)+val
        # print(resp)    
        return resp
    

In [345]:
test=dataOriginal.drop(columns='timestamp')
test=test.sort_values(by='user id')
train,test1=train_test_split(test,train_size=0.2)
x_test=train.drop('rating',axis=1)
y_test=train['rating']
x_test

,user id,item id
70832,736,1089
44878,49,171
83841,870,258
25973,347,756
76127,38,35
...,...,...
52916,350,654
2422,52,1086
74978,779,328
80573,726,763


In [346]:
user_based(195,823,userItem,similaridade)

2.71147436433119

In [347]:
y_test_predict= x_test.apply(lambda x:
           user_based(x['user id'],x['item id'],userItem,similaridade) ,axis=1)
y_test_predict

70832    1.625320
44878    3.046940
83841    3.726314
25973    2.944590
76127    2.591636
           ...   
52916    4.813515
2422     4.160343
74978    3.871505
80573    2.867745
30196    3.157560
Length: 20000, dtype: float64

In [348]:
y_test

70832    1
44878    4
83841    4
25973    2
76127    5
        ..
52916    5
2422     4
74978    4
80573    2
30196    2
Name: rating, Length: 20000, dtype: int64

In [349]:
similaridade

array([[1, 0.9605819645600208, 0.8570746713728922, ...,
        0.9708358225777324, 0.9235049948499491, 0.9297441752941139],
       [0.9605819645600208, 1, 0.9356014857063997, ...,
        0.940225451942229, 0.9484931536348622, 0.9774742538104109],
       [0.8570746713728922, 0.9356014857063997, 1, ...,
        0.9732227014483795, 0.9120093235758657, 0.9899494936611665],
       ...,
       [0.9708358225777324, 0.940225451942229, 0.9732227014483795, ...,
        1, 0.9823024698798778, 0.9607542768511649],
       [0.9235049948499491, 0.9484931536348622, 0.9120093235758657, ...,
        0.9823024698798778, 1, 0.9762865003462128],
       [0.9297441752941139, 0.9774742538104109, 0.9899494936611665, ...,
        0.9607542768511649, 0.9762865003462128, 1]], dtype=object)

In [350]:
mean_absolute_error(y_test,y_test_predict)

0.7334880555237254

In [351]:
np.sqrt(mean_squared_error(y_test,y_test_predict))


0.9302532609350848